# ElasticSearch: _search
"match", "multi_match", "term"


## _search format

In [ ]:
#one match
{
    "query":{
        "match":{
            ....
        }
    }
}

#DSL search
{
    "query":{
        "bool":{
            "must":[
                "match":{...}
            ],
            "must_not":[],
            "should":[],
            "filter":[
                {"match": {...}},
                {"match": {...}}
            ]
        }
    }
}

## scan

In [ ]:
#search: scan all hits
GET orgs-index/_search?pretty
{
 "size":10000,
 "query":{
   "match_all": {}
 },
 "_source": ["ctep_id"]
}  

## exact match
match query is full-text query. The methods:
"match"
"multi_match"
"term"
"terms"

In [ ]:

#search: exact match using 'match'
GET orgs-index/_search?pretty
{
 "size":10000,
 "query":{
   "match": {"ctep_id": "WI137"}
 },
 "_source": ["ctep_id"]
} 

#use funzziness to allow typo input: could be return "Edision" or "Madison"
GET orgs-index/_search?pretty
{
 "size":10000,
 "query":{
   "match": {
     "city_municipality": {
      "query": "medison",
      "fuzziness": "1"
     }
   }
 },
 "_source": ["city_municipality"]
}  


In [ ]:
#search: exact match using 'multi_match'
GET orgs-index/_search?pretty
{
 "size":10000,
 "query":{
   "multi_match": {
      "query": "WI137",
      "fields": ["ctep_id", "id"]
   }
 },
 "_source": ["ctep_id"]
}  

In [ ]:
#search: exact search using "term"
GET orgs-index/_search?pretty
{
 "size":10000,
 "query":{
    "term":{
      "ctep_id":{
        "value": "WI137",
        "case_insensitive": true
      }
    }
 },
 "_source": ["ctep_id"]
}  


## partial search
methods: "wildcard", "regexp", "query_string"

In [ ]:
#search: partial match using wildcard query
GET orgs-index/_search?pretty
{
  "size":10000,
  "query":{
    "wildcard":{
      "ctep_id":{"value":"*WI*"}
    }
  },
  "_source": ["ctep_id"]
}  

In [ ]:
#search: partial match using regexp query
GET orgs-index/_search?pretty
{
  "size":10000,
  "query":{
    "regexp":{
      "ctep_id": {
        "value": "WI[0-9]{3}",
        "case_insensitive": true
      }
    }
  },
  "_source": ["ctep_id"]
} 
  

In [ ]:

#search: partial match using query_string
GET orgs-index/_search?pretty
{
  "size":10000,
  "query":{
    "query_string":{
      "query":"*WI*",
      "fields": ["ctep_id"]
    }
  },
  "_source": ["ctep_id"]
}  

#search: query_string combined with "bool"."must"
GET orgs-index/_search?pretty
{
  "size":10000,
  "query":{
    "bool":{
      "must":[{
        "query_string":{        
          "query": "*37*",
          "fields": ["ctep_id"]
        }
      }],
      "must_not": [],
      "filter": [],
      "should": []
    }
  },
  "_source": ["ctep_id"]
}  

#search: query_string combined with "bool"."must"
GET orgs-index/_search?pretty
{
  "size":10000,
  "query":{
    "bool":{
      "must":[{
        "query_string":{        
          "query": "*37*",
          "fields": ["ctep_id"]
        }
      },
      {
        "query_string":{        
          "query": "16*",
          "fields": ["id"]
        }
      }],
      "must_not": [],
      "filter": [],
      "should": []
    }
  },
  "_source": ["ctep_id", "id"]
}  


## bool query
"must" is equivalent to AND
"must_not" is equivalent to NOT
"should" is equivalent to OR

In [ ]:
GET orgs-index/_search?pretty
{
   "sort":[
    {"name._raw": {"order":"desc"}}
  ],
 "query":{
    "bool": {"must":[],"must_not": [], "should":[],"filter": []}
 }
}

In [ ]:
#must
#where country = United
GET orgs-index/_search?pretty
{
 "size":10000,
 "query":{
   "bool": {
    "must":{"match": {"country": "United"}}
   }
  },
 "_source": ["country", "name"]
}

In [ ]:
#"should"

#when country=United States, city_municipality=Racine or Mddison
GET orgs-index/_search?pretty
{
 "size":10000,
 "query":{
   "bool": {
    "must":[
      {"match": {"country": "United States"}},
      {"match": {"state_province_territory": "WI"}}
    ],
    "should": [
      {"match":{"city_municipality": "Racine"}},
      {"match":{"city_municipality": "Madison"}}
    ]
   }
  }
} 

#minimum_should_match=1
GET orgs-index/_search?pretty
{
 "size":10000,
 "query":{
   "bool": {
    "must":[
      {"match": {"country": "United States"}},
      {"match": {"state_province_territory": "WI"}}
    ],
    "should": [
      {"match":{"city_municipality": "Racine"}},
      {"match":{"city_municipality": "Madison"}}
    ],
    "minimum_should_match": 1
   }
  }
} 

#city_municipality should be Racine or Madison, and va_organization should be false.
GET orgs-index/_search?pretty
{
 "size":10000,
 "query":{
   "bool": {
    "must":[
      {"match": {"country": "United States"}},
      {"match": {"state_province_territory": "WI"}}
    ],
    "should": [{
        "bool":{
          "should": [{"match":{"city_municipality": "Racine"}},
                      {"match":{"city_municipality": "Madison"}}],
          "minimum_should_match": 1
        }
      },{
        "bool":{
          "should": [
            {"match":{"va_organization": false}}
          ],
          "minimum_should_match": 1
        }
    }],
    "minimum_should_match": 2
   }
  }
}


## search with filter
"bool"->"filter"->"exists"
"bool"->"filter"->"term"

In [ ]:
#filter-exists: 

#return all hits where "city_municipality" exist
GET orgs-index/_search?pretty
{
 "size":10000,
 "query":{
   "bool":{
      "filter":{
          "exists":{"field": "city_municipality"}
      }
   }
 }
}  

#return hits where ctep_id is equal to WI137 and "city_municipality" exist 
GET orgs-index/_search?pretty
{
 "size":10000,
 "query":{
   "bool":{
      "must":[
        {"match":{"ctep_id": "WI137"}}
      ],
      "filter":{
          "exists":{"field": "city_municipality"}
      }
   }
 }
}  

In [3]:
#filter-term

#return country="United States"
GET orgs-index/_search?pretty
{
 "size":10000,
 "query":{
    "bool":{
        "filter":[
          {"term": {
              "country": {"value":"United", "case_insensitive":true}
          }},
          {"term": {
            "country": {"value":"States", "case_insensitive":true}
          }}

        ]
    }
 }
}

#return country="United States", and city="Madison"
#Note: all values are low-cases
GET orgs-index/_search?pretty
{
 "size":10000,
 "query":{
    "bool":{
        "filter":[
          {"term": {"country": "united"}},
          {"term": {"country": "states"}},
          {"term": {"city_municipality": "madison"}}
        ]
    }
 }
}

#"term" combined with "range"
GET orgs-index/_search?pretty
{
 "size":10000,
 "query":{
    "bool":{
        "filter":[
          {"term": {"country": "united"}},
          {"term": {"country": "states"}},
          {"range": {"id":{"gte": "200100"}}},
          {"term": {"va_organization": true}}
        ]
    }
 }
}

In [ ]:
#filter-missing
#Note: previous version ES "missing" is deprecated
GET orgs-index/_search?pretty
{
 "size":10000,
 "query":{
    "bool":{
      "must_not":{
        "exists": {"field": "city_municipality"}
      }
    }
 }
}

## other paramester in RESTFUL API

In [ ]:
#pretty
GET orgs-index/_search?pretty

In [ ]:
#_source
#the two ways
GET orgs-index/_search?pretty
{
  "_source": ["id", "name"]
}

#only return organizations.id
GET trials/_search?_source=organizations.id

In [ ]:
#ES page: size and from
#size should be <1000. performane will be poor if size>1000
GET orgs-index/_search
{
  "query":{
    "match_all": {}
  },
  "size": 2,
  "from": 0
  , "_source": ["id"]
}

## multiple match

In [ ]:
GET orgs-index/_search
{
  "query": {
    "bool": {
      "should": [
        {"match": {
          "city_municipality": "racin"
        }},
        {"match": {
          "city_municipality": "madison"
        }}
      ],
      "minimum_should_match": 1
    }
  }
} 